# Tutorial: Classical Shadow Tomography
In this tutorial we will explore shadow tomography using global and pauli ensembles. 
This is implemented via `stim`. All the code should be clear, except perhaps the utility `amplitude_of_state` that outputs $|\langle b| \psi\rangle|$. 

In [14]:
import stim 
import numpy as np 
from typing import List, Tuple

def amplitude_of_state(state: stim.TableauSimulator,desired_state: List[bool]) -> complex:
  n = state.num_qubits
  copy = state.copy()
  num_random = 0
  for q in range(n)[::-1]:
    desired_bit = desired_state[q]
    z = copy.peek_z(q)
    is_random = z == 0
    forced_bit = z == -1
    if is_random:
      num_random += 1
    elif desired_bit != forced_bit:
      return 0
    copy.postselect_z(q, desired_value=desired_bit)
  magnitude = 2**(-(num_random / 2))
  return magnitude

## Implementation
Let us define the state of interest: a noisy GHZ state. The implementation follows the trivial, depth $N$ quantum circuit (see Lecture Notes). As noise model, we consider a depolarizing channel randomly acting on one qubit in the GHZ state with probability `probability_fault`.

In [2]:
def prepare_state_ghz(N,probability_fault=0):
  psi = stim.TableauSimulator()
  psi.set_num_qubits(N)
  psi.h(0)
  for i in range(1,N):
    psi.cx(0,i)
  psi.depolarize1(np.random.randint(N),p=probability_fault)
  return psi

#example for N=4 qubit and without noise
prepare_state_ghz(4,0).current_inverse_tableau().inverse().to_circuit().diagram()

q0: -H-@-@-@-
       | | |
q1: ---X-|-|-
         | |
q2: -----X-|-
           |
q3: -------X-

We will consider two type of snapshots: (i) those obtained from the global Clifford ensemble $C\in\mathcal{C}_N$, and (ii) those obtained as a tensor product $C=C_1\otimes C_2\otimes \cdots \otimes C_N$ with each $C_i\in \mathcal{C}_1$ a single qubit random Clifford gate.
For the global, we recall the inverse channel generate the classical snapshot 
$$\hat{\rho} = (2^N+1) C^\dagger|b\rangle\langle b| C - \mathbb{I}\;,$$
for a given realization of the Clifford unitary $C$ and of the measurement result $|b\rangle$. 

For the local case, we have instead
$$\hat{\rho} = \bigotimes_{r=1}^N \left[3 C_r^\dagger|b_r\rangle\langle b_r| C_r - \mathbb{I}\right]\;,$$
where $C_r$ are the single qubit Clifford gates, and $b_r$ is the binary measurement result at site $r$. 

The function `global_classical_snapshot` gives a sample of pairs $\{(b^{(j)},C^{(j)})\}_{j=1,\dots,N_\mathrm{sample}}$. Similarly, the function `local_classical_snapshot` gives a sample of pairs $\{(b^{(j)},\{C_r^{(j)})\}_{r=1,\dots,N}\}_{j=1,\dots,N_\mathrm{sample}}$, with the second element of each pair being the list of local Clifford gate.

In [3]:
def global_classical_snapshot(psi: stim.TableauSimulator, sample_size: int =50) -> List[Tuple[List,stim.TableauSimulator]]:
  N = psi.num_qubits
  targets = list(range(N))
  pairs = []
  for _ in range(sample_size):
    U = stim.Tableau.random(N)
    phi = psi.copy()
    phi.do_tableau(U,targets)
    b = phi.measure_many(*targets)
    pairs.append((b,U))
  return pairs

def local_classical_snapshot(psi: stim.TableauSimulator, sample_size: int =50) -> List[Tuple[List,List[stim.TableauSimulator]]]:
  N = psi.num_qubits
  targets = list(range(N))
  pairs = []
  for _ in range(sample_size):
    phi = psi.copy()
    Us = [] 
    for r in range(N):
      U = stim.Tableau.random(1)
      Us.append(U)
      phi.do_tableau(U,[r])
    b = phi.measure_many(*targets)
    pairs.append((b,Us))
  return pairs

The classical snapshot allow us to postprocess the data to compute expectation values of observables $\mathrm{tr}(O\rho) = \mathbb{E}[\mathrm{tr}(O\hat{\rho})]$. 
For the fidelity with a given reference state $|phi\rangle$, defined as $F=|\langle \phi | \psi\rangle|^2$ we will use the global estimator. 


For a sample of $M$ snapshots, we consider the estimator

$\displaystyle F_\mathrm{est}= \frac{1}{M} \sum_{m=1}^M \left[(2^N+1) |\langle \phi | U^\dagger |b\rangle|^2 - 1 \right] = \frac{1}{M} \sum_{m=1}^M \left((2^N+1) |\langle b | U |\phi\rangle|^2 - 1 \right)$.

In [4]:
def global_fidelity_estimator(psi_target: stim.TableauSimulator,pairs):
  F_estimator = 0 
  N = psi_target.num_qubits
  for (b,U) in pairs:
    phi = psi_target.copy()
    phi.do_tableau(U,range(N))
    F_estimator+= (2**N+1)*amplitude_of_state(phi,b)**2 -1 
  return F_estimator/len(pairs) 

For a Pauli string observable $P$, we will consider both global and local estimators, respectively reading

$\displaystyle O_\mathrm{est}= \frac{1}{M} \sum_{m=1}^M \left[(2^N+1) \langle b |U P U^\dagger |b\rangle\right]\;,$ and 

$\displaystyle O_\mathrm{est}= \frac{1}{M} \sum_{m=1}^M \prod_{r=1}^N \left[(2^N+1) \langle b_r |C_r P_r C_r^\dagger |b_r\rangle - \mathrm{tr}(P_r)\right]\;,$

In [5]:
def global_observable_estimator(pauli: stim.PauliString, pairs: List[Tuple[List,stim.Tableau]]):
  O_estimator = 0 
  for (b,U) in pairs:
    N = len(b)
    phi = stim.TableauSimulator()
    phi.set_num_qubits(N)
    phi.x(*np.nonzero(b)[0])
    phi.do_tableau(U.inverse(),range(N))
    O_estimator += (2**N+1)*phi.peek_observable_expectation(pauli)
  return O_estimator/len(pairs) 

def local_observable_estimator(pauli: stim.PauliString, pairs: List[Tuple[List,List[stim.Tableau]]]):
  O_estimator = 0 
  for (b,Us) in pairs:
    O_local = 1 
    for (bi,U,P) in zip(b,Us,pauli):
      phi = stim.TableauSimulator()
      if (bi):
        phi.x(0)
      phi.do_tableau(U.inverse(),[0])
      if (P):
        O_local*=3*phi.peek_observable_expectation(stim.PauliString([P]))
    O_estimator+= O_local
  return O_estimator/len(pairs)

We collect all the previous functions to `fidelity_shadow_tomography`, `global_observable_shadow_tomography`, and `local_observable_shadow_tomography`. 

In [6]:
def fidelity_shadow_tomography(psi_input: stim.TableauSimulator, psi_target: stim.TableauSimulator, sample_size: int =100, number_shots: int =100,) -> Tuple[float,float]:
  F_estimators = []
  for m in range(number_shots):
    snapshot=global_classical_snapshot(psi_input,sample_size=sample_size)
    F_estimators.append(global_fidelity_estimator(psi_target,snapshot))
  return (np.mean(F_estimators), np.std(F_estimators)/np.sqrt(sample_size-1))

def global_observable_shadow_tomography(psi_input: stim.TableauSimulator, pauli: stim.PauliString, sample_size: int =100, number_shots: int =100,) -> Tuple[float,float]:
  O_estimators = []
  for m in range(number_shots):
    snapshot=global_classical_snapshot(psi_input,sample_size=sample_size)
    O_estimators.append(global_observable_estimator(pauli,snapshot))
  return (np.mean(O_estimators), np.std(O_estimators)/np.sqrt(sample_size-1))

def local_observable_shadow_tomography(psi_input: stim.TableauSimulator, pauli: stim.PauliString, sample_size: int=100, number_shots: int=100,) -> Tuple[float,float]:
  O_estimators = []
  for m in range(number_shots):
    snapshot=local_classical_snapshot(psi_input,sample_size=sample_size)
    O_estimators.append(local_observable_estimator(pauli,snapshot))
  return (np.mean(O_estimators), np.std(O_estimators)/np.sqrt(sample_size-1))

## Analysis
Now we are in position to test the shadow tomography protocol. For the fidelity, we consider the global Clifford transformation. The shadow norm is guaranteed to be $\| |\phi\rangle\langle \phi|\|_\mathrm{shadow} \le 3 \mathrm{tr}(|\phi\rangle\langle \phi|^2) = 3.$
In particular, increasing system size, the error should fluctuate around the same value (fixed by the choise of $M=N_\mathrm{sample}$ and $K=N_\mathrm{shots}$).

In [13]:
for N in range(2,14,2):
  psi_ideal= prepare_state_ghz(N,0)
  psi_input = psi_ideal.copy()
  mean, err = fidelity_shadow_tomography(psi_input,psi_ideal,)
  print(N,mean,err)

2 1.004375 0.010538492155863282
4 0.9941 0.012630381641266444
6 0.978640625 0.014367008665453412
8 1.0127316406250002 0.014410527988301511
10 0.9939453125 0.014017282436437541
12 0.997487548828125 0.01290957409535943


Viceversa, for Pauli expectation values, the shadow norm explodes:
$\| P\|_\mathrm{shadow} \le 3 \mathrm{tr}(P^2) = 3\; \mathrm{tr}(\mathbb{I}) = 3 \times 2^N$.
So, increasing system size, the error will increase exponentially fast, *de facto* killing the employability of the method.

In [8]:
print("Local pauli string")
for N in range(2,14,2):
  psi_ideal= prepare_state_ghz(N,0)
  psi_input = psi_ideal.copy()
  pauli = stim.PauliString([3,3]+[0]*(N-2))
  mean, err = global_observable_shadow_tomography(psi_input,pauli,)
  print(N,mean,err)

print("Global pauli string")
for N in range(2,14,2):
  psi_ideal= prepare_state_ghz(N,0)
  psi_input = psi_ideal.copy()
  pauli = stim.PauliString([1]*N)
  mean, err = global_observable_shadow_tomography(psi_input,pauli,)
  print(N,mean,err)

Local pauli string
2 0.9655000000000002 0.02000877332824304
4 1.0471999999999997 0.040048980112866894
6 1.079 0.09015189202875133
8 1.2335999999999998 0.17319233892590707
10 1.1275 0.382550171661684
12 3.2776 1.1170877100385588
Global pauli string
2 1.0085000000000002 0.021118413144395432
4 1.0097999999999998 0.03864680263017051
6 1.0725000000000002 0.08975098828976702
8 0.8480999999999997 0.15945384658865772
10 1.025 0.3090491281922077
12 0.4097 0.4097


Using instead local Clifford operations, the shadow tomography efficacy is fixed by the weight $w(P)$ of the Pauli string (number of Pauli matrices in the string that are non-identity): $\| P\|_\mathrm{shadow} \le 4^{w(P)}\|P\|_\infty^2 = 4^{w(P)}$.
So, we expect the value to be constant for a given Pauli weight string, and to increase exponentially with system size for any Pauli string sufficiently distributed along the system. 

In [9]:
print("Local pauli string. The exact value of reference for the expectation value is 1.")
for N in range(2,14,2):
  psi_ideal= prepare_state_ghz(N,0)
  psi_input = psi_ideal.copy()
  pauli = stim.PauliString([3,3]+[0]*(N-2))
  mean, err = local_observable_shadow_tomography(psi_input,pauli,)
  print(N,mean,err)

print("Global pauli string. The exact value of reference for the expectation value is 1.")
for N in range(2,14,2):
  psi_ideal= prepare_state_ghz(N,0)
  psi_input = psi_ideal.copy()
  pauli = stim.PauliString([1]*N)
  mean, err = local_observable_shadow_tomography(psi_input,pauli,)
  print(N,mean,err)

Local pauli string. The exact value of reference for the expectation value is 1.
2 1.0125000000000002 0.027831881653306093
4 0.9837 0.02650315590127471
6 1.035 0.029240383034426887
8 1.0116 0.02526614695235864
10 0.9810000000000001 0.030392881941784865
12 0.9828 0.025860183504938457
Global pauli string. The exact value of reference for the expectation value is 1.
2 1.0197 0.02752682988715489
4 0.9234000000000003 0.07724763014899787
6 0.9477000000000001 0.28668008619048135
8 1.3122 0.9231674348281964
10 0.0 0.0
12 0.0 0.0


Finally, we consider the case our state is faulty. This is exactly the reason why often we consider $|\psi_\mathrm{input}\rangle$ as the target for the fidelity. In general, because of noise $|\psi_\mathrm{input}\rangle\langle \psi_\mathrm{input}| = |\psi_\mathrm{target}\rangle \langle \psi_\mathrm{target}| + \varepsilon \delta \rho$. 

In [10]:
psi_ideal= prepare_state_ghz(4,0)
probability_fault = 0.05 # 5% error

target_benchmark = []
target_benchmark_error = []
for _ in range(100):
  psi_noisy = prepare_state_ghz(4,probability_fault=probability_fault)
  mean, err = fidelity_shadow_tomography(psi_noisy,psi_target=psi_ideal)
  target_benchmark.append(mean)
  target_benchmark_error.append(err)

In [12]:
print("Ideal fidelity 1, vs noisy case:")
print(np.mean(target_benchmark))

Ideal fidelity 1, vs noisy case:
0.9607757499999999
